In [ ]:
############# DOWNLOAD DATA ##################
from sklearn import datasets
iris = datasets.load_iris()

X = iris.data
y = iris.target

In [ ]:
############## TRAIN AND VALIDATE MODEL ##################################

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
# Split data into 90% train and 10% test subsets
x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

# Normalizing the data
x_train = x_train/255.0
x_test = x_test/255.0

depth = 50
n_estimators = 20
random_state = 1

model_clf = RandomForestClassifier(n_estimators=n_estimators, max_depth=depth, random_state=random_state)

# Train the Random Forest algorithm
model_clf.fit(x_train, y_train)

y_pred = model_clf.predict(x_test)

total = len(y_pred)
wrong = 0
for pred, truth in zip(y_pred, y_test):
    wrong += 1 if pred != truth else 0

accuracy = (total - wrong)/total
print(accuracy)

In [ ]:
%%writefile hooks.py

import process

def input_fn(X):
    return process.pre_process(X)

def output_fn(Y):
    return Y

In [ ]:
################## LOG MODEL ###########################
import mlflow

mlflow.set_tracking_uri("http://model-repository.stg.dreamplug.net/")
mlflow.set_experiment("ds-test-1")
model = None
with mlflow.start_run() as run:
    mlflow.log_param("n_estimators", n_estimators)
    mlflow.log_param("max_depth", depth)
    mlflow.log_param("random_state", random_state)
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_artifacts("hooks.py", "model")
    mlflow.sklearn.log_model(model_clf, "model", 
                             registered_model_name="TEST-DYNAMIC-HOOKS", 
                             extra_pip_requirements=['git+ssh://git@github.com/nikunjsharma92/hooks-test@v2'] # pip repository from github. imports "process" lib
                            )